In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
spark = SparkContext(conf=conf).getOrCreate()

24/12/03 16:25:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 16:25:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import os
directory = os.path.join(os.getcwd(), "data")
filename = "restaurant_reviews.csv"
filepath = os.path.join(directory, filename)

lines = spark.textFile("file:///"+filepath.replace("\\", "/"))
lines

file:////home/lab15/src/src/data/restaurant_reviews.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [3]:
lines.take(15)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23,']

In [4]:
header = lines.first()
filtered_lines = lines.filter(lambda row : row != header)

In [5]:
filtered_lines.take(5)

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,']

In [6]:
def parse(row):
    fields = row.split(",")
    category = fields[2]
    reviews = int(fields[3])
    
    return category, reviews

In [7]:
parse('0,짜장면,중식,125,')

('중식', 125)

RDD내 모든 row에 대해 parse()를 적용한 다음, map()추출

In [8]:
category_reviews = filtered_lines.map(parse)
category_reviews

PythonRDD[5] at RDD at PythonRDD.scala:53

In [9]:
category_reviews.take(10)

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [10]:
category_reviews.take(5)

[('중식', 125), ('중식', 235), ('분식', 32), ('분식', 534), ('일식', 223)]

In [11]:
cate_review_cnt = category_reviews.mapValues(lambda x: (x, 1))
cate_review_cnt.take(3)

[('중식', (125, 1)), ('중식', (235, 1)), ('분식', (32, 1))]

'중식' (125,1)
첫번째 중식 > (125,1) -> x , 두번째 중식 > (235,1) -> y
리뷰의 갯수의 합 = x[0] + y[0]
건수 합 = x[1] + y[1]

In [12]:
reduce_rdd = cate_review_cnt.reduceByKey(lambda x,y : (x[0]+ y[0], x[1]+y[1]))
reduce_rdd.take

<bound method RDD.take of PythonRDD[13] at RDD at PythonRDD.scala:53>

In [13]:
reduce_rdd.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [14]:
# 평균
reduce_rdd_avg = reduce_rdd.mapValues(lambda x: x[0] / x[1])

In [15]:
reduce_rdd_avg.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]